## Collaborative Filtering for  Job Recommendation
Collaborative filtering uses user data to suggest products they're likely to be interested in

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Prajwal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Impoting Libraries

In [2]:

import numpy as np
import pandas as pd
import seaborn as sns
import joblib
import matplotlib as map
import matplotlib.pyplot as plt
import re
from nltk.tokenize import word_tokenize
import nltk
import operator
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 


#To ignore warnings.
import warnings
warnings.filterwarnings('ignore')

#saving model
#from sklearn.externals import joblib

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Prajwal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Loading the Dataset

In [3]:
df_all_offers = pd.read_csv(r'C:\Users\Prajwal\Desktop\Vyzen Assessment\Job Recommendation\datasets\ALL_Offers.csv')
df_offers = pd.read_csv(r'C:\Users\Prajwal\Desktop\Vyzen Assessment\Job Recommendation\datasets\offers.csv')    
df_org = pd.read_csv(r'C:\Users\Prajwal\Desktop\Vyzen Assessment\Job Recommendation\datasets\organizations.csv')

### Data Analysis

In [4]:
df_all_offers.head()

,posted_by_id,status,city,job_title,organization_id,contracts,description,skills
0,1,published,Bondues,Stagiaire (H/F) Assistant Marketing Nutritionnel,1,Stage6mois,Entreprise Le Groupe LACTALIS (près de ...,{'skills': {'communication': 1}}
1,2,published,Rhône,Expert process traitement thermique h/f,2,CDI,Entreprise Filiale européenne du groupe...,"{'skills': {'communication': 1, 'consulting': ..."
2,3,published,Paris,Charge d'operation grands comptes h/f,3,CDI,"Entreprise Notre client, première socié...","{'skills': {'collaboration': 1, 'reporting': 1..."
3,4,published,Rueil-Malmaison,Ingénieur Génie Civil stage F/H,4,Stage6mois,Entreprise Acteur mondial et leader eur...,"{'skills': {'collaboration': 1, 'ensemble': 1,..."
4,5,published,Levallois-Perret,Chargé pricing H/F,5,CDI,"Entreprise ALLIANCE AUTOMOTIVE France, ...","{'skills': {'collaboration': 1, 'leadership': 1}}"


In [5]:
#Number of Rows in the dataset
print("Number of Rows are:",df_all_offers.shape[0])

Number of Rows are: 8867


In [6]:

#Number of Columns/features in dataset
print("Number of Columns are:",df_all_offers.shape[1])

Number of Columns are: 8


In [7]:
#List of Available features in dataset
print("Available Features are:",df_all_offers.columns.tolist())

Available Features are: ['posted_by_id', 'status', 'city', 'job_title', 'organization_id', 'contracts', 'description', 'skills']


In [8]:
# Let's checkout which feature contains which kind of values i.e int /float /etc or what is the data type of values.
print(df_all_offers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8867 entries, 0 to 8866
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   posted_by_id     8867 non-null   int64 
 1   status           8867 non-null   object
 2   city             8867 non-null   object
 3   job_title        8867 non-null   object
 4   organization_id  8867 non-null   int64 
 5   contracts        8867 non-null   object
 6   description      8867 non-null   object
 7   skills           8867 non-null   object
dtypes: int64(2), object(6)
memory usage: 554.3+ KB
None


In [9]:
#Checking is their any missing values are available in the dataset

print("Missing values:\n ",df_all_offers.isnull().sum())

Missing values:
  posted_by_id       0
status             0
city               0
job_title          0
organization_id    0
contracts          0
description        0
skills             0
dtype: int64


In [10]:
# check for duplicate values
df_all_offers.duplicated().sum()

141

In [11]:
#checing number of unique values available in each column
df_all_offers.nunique()

posted_by_id       1345
status                1
city               1020
job_title          6770
organization_id    1345
contracts            52
description        8707
skills             2950
dtype: int64

In [12]:
#Let's understand the data
df_offers.head()

,user_id,status,city,job_title,organization_id,contracts,description,skills
0,55,published,HOMERVILLE,Medical Receptionist,0,AL,'Lorem ipsum dolor sit amet consectetur adipis...,"{'skills': {'training': 1, 'certifications': 1..."
1,90,published,DEMOTTE,Clinical Research Associate,1,AL,'Lorem ipsum dolor sit amet consectetur adipis...,"{'skills': {'hardware': 2, 'communication': 1,..."
2,2,published,CARY,Territory Sales Representative,2,AL,'Lorem ipsum dolor sit amet consectetur adipis...,"{'skills': {'communication': 4, 'e-commerce': 1}}"
3,3,published,SKOWHEGAN,Bus Driver,3,AL,'Lorem ipsum dolor sit amet consectetur adipis...,"{'skills': {'mobile': 1, 'communication': 2, '..."
4,4,published,HOLLISTER,Foreign Service Officer,4,AL,'Lorem ipsum dolor sit amet consectetur adipis...,"{'skills': {'training': 4, 'designer': 1, 'pro..."


In [13]:
#Number of Rows in the dataset
print("Number of Rows are:",df_offers.shape[0])

Number of Rows are: 252


In [14]:
#Number of Columns/features in dataset
print("Number of Columns are:",df_offers.shape[1])

Number of Columns are: 8


In [15]:
#List of Available features in dataset
print("Available Features are:",df_offers.columns.tolist())

Available Features are: ['user_id', 'status', 'city', 'job_title', 'organization_id', 'contracts', 'description', 'skills']


In [16]:

# Let's checkout which feature contains which kind of values i.e int /float /etc or what is the data type of values.
print(df_offers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          252 non-null    int64 
 1   status           252 non-null    object
 2   city             252 non-null    object
 3   job_title        252 non-null    object
 4   organization_id  252 non-null    int64 
 5   contracts        252 non-null    object
 6   description      252 non-null    object
 7   skills           252 non-null    object
dtypes: int64(2), object(6)
memory usage: 15.9+ KB
None


In [17]:
#Checking is their any missing values are available in the dataset

print("Missing values:\n ",df_offers.isnull().sum())

Missing values:
  user_id            0
status             0
city               0
job_title          0
organization_id    0
contracts          0
description        0
skills             0
dtype: int64


In [18]:
# check for duplicate values
df_offers.duplicated().sum()

0

In [19]:
#Let's understand the data
df_org.head()

,organization_id,company_name,email,address,size,sectors
0,0,BARON & DECK,John.C.Maher@spambob.com,1018 1ST AVE APT 1229 # 7A,MC,PT
1,1,HILLSIDE TERRACE APTS,Shelley.M.Hamer@mailinator.com,10051 SANDRIDGE RD,BG,VA
2,2,GEORGE B LEWIS APTS,Tracie.M.Vargas@dodgit.com,10964 54TH ST SE,MC,GD
3,3,F A A AIRPORT,John.P.Stephens@pookmail.com,10826 WALNUT ST,MD,MA
4,4,BON JOUR ANSWERING,Nicholas.J.Fisher@spambob.com,101 JACKS LN,ST,GD


In [20]:
#Number of Rows in the dataset
print("Number of Rows are:",df_org.shape[0])

Number of Rows are: 201


In [21]:
#Number of Columns/features in dataset
print("Number of Columns are:",df_org.shape[1])

Number of Columns are: 6


In [22]:
#List of Available features in dataset
print("Available Features are:",df_org.columns.tolist())

Available Features are: ['organization_id', 'company_name', 'email', 'address', 'size', 'sectors']


In [23]:
# Let's checkout which feature contains which kind of values i.e int /float /etc or what is the data type of values.
print(df_offers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          252 non-null    int64 
 1   status           252 non-null    object
 2   city             252 non-null    object
 3   job_title        252 non-null    object
 4   organization_id  252 non-null    int64 
 5   contracts        252 non-null    object
 6   description      252 non-null    object
 7   skills           252 non-null    object
dtypes: int64(2), object(6)
memory usage: 15.9+ KB
None


In [24]:
#Checking is their any missing values are available in the dataset

print("Missing values:\n ",df_org.isnull().sum())

Missing values:
  organization_id    0
company_name       0
email              0
address            0
size               0
sectors            0
dtype: int64


In [25]:
#checing number of unique values available in each column
df_org.nunique()

organization_id    201
company_name       106
email              201
address            201
size                 4
sectors              5
dtype: int64

In [26]:
#Combining the dataset Offers.csv and Org.csv
df_combined1=pd.merge(df_offers,df_org)
df_combined1.head()

,user_id,status,city,job_title,organization_id,contracts,description,skills,company_name,email,address,size,sectors
0,55,published,HOMERVILLE,Medical Receptionist,0,AL,'Lorem ipsum dolor sit amet consectetur adipis...,"{'skills': {'training': 1, 'certifications': 1...",BARON & DECK,John.C.Maher@spambob.com,1018 1ST AVE APT 1229 # 7A,MC,PT
1,90,published,DEMOTTE,Clinical Research Associate,1,AL,'Lorem ipsum dolor sit amet consectetur adipis...,"{'skills': {'hardware': 2, 'communication': 1,...",HILLSIDE TERRACE APTS,Shelley.M.Hamer@mailinator.com,10051 SANDRIDGE RD,BG,VA
2,2,published,CARY,Territory Sales Representative,2,AL,'Lorem ipsum dolor sit amet consectetur adipis...,"{'skills': {'communication': 4, 'e-commerce': 1}}",GEORGE B LEWIS APTS,Tracie.M.Vargas@dodgit.com,10964 54TH ST SE,MC,GD
3,3,published,SKOWHEGAN,Bus Driver,3,AL,'Lorem ipsum dolor sit amet consectetur adipis...,"{'skills': {'mobile': 1, 'communication': 2, '...",F A A AIRPORT,John.P.Stephens@pookmail.com,10826 WALNUT ST,MD,MA
4,4,published,HOLLISTER,Foreign Service Officer,4,AL,'Lorem ipsum dolor sit amet consectetur adipis...,"{'skills': {'training': 4, 'designer': 1, 'pro...",BON JOUR ANSWERING,Nicholas.J.Fisher@spambob.com,101 JACKS LN,ST,GD


In [27]:
#Number of Rows in the dataset
print("Number of Rows are:",df_combined1.shape[0])
#Number of Columns/features in dataset
print("Number of Columns are:",df_combined1.shape[1])

Number of Rows are: 252
Number of Columns are: 13


In [28]:
#Combining the Datasets All_offers.csv and Org.csv
df_combined2=pd.merge(df_all_offers,df_org,on='organization_id')
df_combined2.head()

,posted_by_id,status,city,job_title,organization_id,contracts,description,skills,company_name,email,address,size,sectors
0,1,published,Bondues,Stagiaire (H/F) Assistant Marketing Nutritionnel,1,Stage6mois,Entreprise Le Groupe LACTALIS (près de ...,{'skills': {'communication': 1}},HILLSIDE TERRACE APTS,Shelley.M.Hamer@mailinator.com,10051 SANDRIDGE RD,BG,VA
1,2,published,Rhône,Expert process traitement thermique h/f,2,CDI,Entreprise Filiale européenne du groupe...,"{'skills': {'communication': 1, 'consulting': ...",GEORGE B LEWIS APTS,Tracie.M.Vargas@dodgit.com,10964 54TH ST SE,MC,GD
2,3,published,Paris,Charge d'operation grands comptes h/f,3,CDI,"Entreprise Notre client, première socié...","{'skills': {'collaboration': 1, 'reporting': 1...",F A A AIRPORT,John.P.Stephens@pookmail.com,10826 WALNUT ST,MD,MA
3,4,published,Rueil-Malmaison,Ingénieur Génie Civil stage F/H,4,Stage6mois,Entreprise Acteur mondial et leader eur...,"{'skills': {'collaboration': 1, 'ensemble': 1,...",BON JOUR ANSWERING,Nicholas.J.Fisher@spambob.com,101 JACKS LN,ST,GD
4,5,published,Levallois-Perret,Chargé pricing H/F,5,CDI,"Entreprise ALLIANCE AUTOMOTIVE France, ...","{'skills': {'collaboration': 1, 'leadership': 1}}",COLLINS POINT,Kenneth.A.Salazar@mailinator.com,10694 DOW RD,ST,MA


In [29]:
#Number of Rows in the dataset
print("Number of Rows are:",df_combined2.shape[0])
#Number of Columns/features in dataset
print("Number of Columns are:",df_combined2.shape[1])

Number of Rows are: 5087
Number of Columns are: 13


In [30]:
df_combined3=pd.merge(df_all_offers,df_org,on='organization_id',how='outer')
print("Number of Rows are:",df_combined3.shape[0])

Number of Rows are: 8868


In [31]:
df_combined3.isnull().sum()

posted_by_id          1
status                1
city                  1
job_title             1
organization_id       0
contracts             1
description           1
skills                1
company_name       3780
email              3780
address            3780
size               3780
sectors            3780
dtype: int64

In [32]:
#Combining the Datasets All_offers.csv and Org.csv
df_combined1['contracts'].unique()

array(['AL'], dtype=object)

In [33]:
jobs = df_combined1['job_title'].unique()
jobs

array(['Medical Receptionist', 'Clinical Research Associate',
       'Territory Sales Representative', 'Bus Driver',
       'Foreign Service Officer', 'Sales Analyst', 'Strategy Consultant?',
       'Cost Engineer', 'Solutions Consultant - Electrical Engineering',
       'Power Regulator', 'Conservation Technician',
       'Manager of Engineering', 'Forecast Systems Manager',
       'Water Services Technician',
       'Senior Analyst Retail Analytics and Reporting',
       'Total Quality Management Director', 'HVAC Mechanic',
       'Cognos Business Development Architect', 'School Social Worker',
       'Treasurer', 'Manufacturing Customer Service Representative',
       'Quality Assurance Manager', 'Night Auditor',
       'Business Intelligence Developer', 'Solid Waste Field Technician',
       'Healthcare Consultant', 'Organic Lab Worker',
       'Compliance Engineer',
       'Chief of Career Pathways and Integrated Learning',
       'Assistant Coach', 'IT Support Staff', 'Meeting Sp

In [34]:
len(jobs)

222

In [35]:
newdf = df_combined1[['user_id', 'job_title','skills']]
newdf

,user_id,job_title,skills
0,55,Medical Receptionist,"{'skills': {'training': 1, 'certifications': 1..."
1,90,Clinical Research Associate,"{'skills': {'hardware': 2, 'communication': 1,..."
2,2,Territory Sales Representative,"{'skills': {'communication': 4, 'e-commerce': 1}}"
3,3,Bus Driver,"{'skills': {'mobile': 1, 'communication': 2, '..."
4,4,Foreign Service Officer,"{'skills': {'training': 4, 'designer': 1, 'pro..."
...,...,...,...
247,74,Assistant General Manager,"{'skills': {'http': 2, 'postfix': 2, 'mobile':..."
248,8,Operations Agent,"{'skills': {'communication': 1, 'reporting': 2}}"
249,13,Event Coordinator,"{'skills': {'reporting': 2, 'ai': 5, 'communic..."
250,17,Mutual Fund Analyst,"{'skills': {'mobile': 2, 'communication': 1, '..."


In [36]:
def delete_spec_chars(input): 
    #function to delete special characters

    regex = r'[^a-zA-Z0-9\s]'
    output = re.sub(regex,'',input)
    return output

#creating list of dictionary to store the job and it's corresponding skills
skills = []
new_skills=[]
for j in jobs:
    #traversing through job and comparing it with job title in dataframe if both are same then adding their skills.

    #creating inner list to store the skills of one job
    d=[]
    inner_list=[]
    for i, row in newdf.iterrows():
        if(row['job_title']==j):
            d.append(row['skills'])
    print(d)

  
    #removing all the special characters from that
    doc = delete_spec_chars(str(d))

    #removing digits from that
    doc = re.sub(r'\d+','',doc)

    #removing skills word from that
    doc=re.sub(r'skills','',doc)

    #Applying word tokenization to extract tokens
    tokens = word_tokenize(doc) 
    print("document is: ")
    print(doc)
    print(" tokens are: ")

    tokens= [word.lower() for word in tokens]
    print(tokens)
    
    #To remove duplicate skills 
    tokens=list(set(tokens))

    #Adding that in skills dicitonary for one job
    skills.append(tokens)

    inner_list.append(doc)
    new_skills.append(inner_list)

#   skillss.append(newdf.loc[newdf['job_title'] == j]['skills'])
  
print(skills)

["{'skills': {'training': 1, 'certifications': 1, 'http': 1, 'processing': 1, 'implementation': 1}}", "{'skills': {'training': 1, 'certifications': 1, 'http': 1, 'processing': 1, 'implementation': 1, 'documentation': 2, 'programming': 1, 'prototyping': 1, 'tech support': 1, 'penetration testing': 1, 'data processing': 1, 'technical': 8, 'networks': 1}}"]
document is: 
 training  certifications  http  processing  implementation   training  certifications  http  processing  implementation  documentation  programming  prototyping  tech support  penetration testing  data processing  technical  networks 
 tokens are: 
['training', 'certifications', 'http', 'processing', 'implementation', 'training', 'certifications', 'http', 'processing', 'implementation', 'documentation', 'programming', 'prototyping', 'tech', 'support', 'penetration', 'testing', 'data', 'processing', 'technical', 'networks']
["{'skills': {'hardware': 2, 'communication': 1, 'ensemble': 1}}"]
document is: 
 hardware  communi

In [37]:
skills[0]

['training',
 'data',
 'certifications',
 'implementation',
 'penetration',
 'processing',
 'tech',
 'programming',
 'documentation',
 'technical',
 'http',
 'support',
 'networks',
 'testing',
 'prototyping']

In [38]:
#retreiving all the skills from skills set and storing them in total_skills list.
total_skills=[]
for val in skills:
    for keys in val:
        if keys not in total_skills:
            total_skills.append(keys)
        else:
            continue

total_skills=list(set(total_skills))
print(total_skills)

print("Total number of available skills are: ", len(total_skills))

['ensemble', 'computing', 'electronics', 'wimax', 'teamwork', 'collaboration', 'volleyball', 'nodejs', 'project', 'mining', 'customer', 'internet', 'services', 'social', 'website', 'postfix', 'energy', 'software', 'html', 'desk', 'development', 'design', 'backend', 'modeling', 'powershell', 'server', 'time', 'consulting', 'linux', 'wordpress', 'go', 'telecom', 'reporting', 'ajax', 'jquery', 'spark', 'technologies', 'networks', 'thinking', 'bootstrap', 'service', 'certifications', 'implementation', 'sql', 'solving', 'vbnet', 'python', 'aspnet', 'statistical', 'visio', 'pascal', 'swing', 'digital', 'coding', 'critical', 'sencha', 'matlab', 'management', 'prototyping', 'media', 'processing', 'prestashop', 'ai', 'renewable', 'ecommerce', 'dom', 'designer', 'touch', 'debugging', 'learning', 'windows', 'collection', 'penetration', 'organizational', 'javascript', 'problem', 'priorities', 'coaching', 'assembly', 'innovation', 'manager', 'graphic', 'less', 'mathematics', 'mentoring', 'technical

In [39]:
#To create dataframe, creating columns of that

column_headers=['Jobs']

for val in total_skills:
    column_headers.append(val)

print(column_headers)
print("total number of columns are: ", len(column_headers))

['Jobs', 'ensemble', 'computing', 'electronics', 'wimax', 'teamwork', 'collaboration', 'volleyball', 'nodejs', 'project', 'mining', 'customer', 'internet', 'services', 'social', 'website', 'postfix', 'energy', 'software', 'html', 'desk', 'development', 'design', 'backend', 'modeling', 'powershell', 'server', 'time', 'consulting', 'linux', 'wordpress', 'go', 'telecom', 'reporting', 'ajax', 'jquery', 'spark', 'technologies', 'networks', 'thinking', 'bootstrap', 'service', 'certifications', 'implementation', 'sql', 'solving', 'vbnet', 'python', 'aspnet', 'statistical', 'visio', 'pascal', 'swing', 'digital', 'coding', 'critical', 'sencha', 'matlab', 'management', 'prototyping', 'media', 'processing', 'prestashop', 'ai', 'renewable', 'ecommerce', 'dom', 'designer', 'touch', 'debugging', 'learning', 'windows', 'collection', 'penetration', 'organizational', 'javascript', 'problem', 'priorities', 'coaching', 'assembly', 'innovation', 'manager', 'graphic', 'less', 'mathematics', 'mentoring', 't

In [40]:
#creating list to insert it in dataframe.
data_list = []

for index, obj in enumerate(skills):
    inner_list=[]
    # print(obj)

    # print(index)
    # inner_list.append(jobs[index])

    for val in column_headers:
        if val=='Jobs':
            inner_list.append(jobs[index])
        else:
            if val in obj:
                inner_list.append(1)
            else:
                inner_list.append(0)

    data_list.append(inner_list)

print(data_list)

#Finding length to check if skills set is added for each job or not
print("len is", len(data_list))

[['Medical Receptionist', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], ['Clinical Research Associate', 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ['Territor

In [41]:
print(data_list[0])

['Medical Receptionist', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [42]:
#converting into dataframe:

meta_data = pd.DataFrame(data_list,columns=column_headers)
meta_data

,Jobs,ensemble,computing,electronics,wimax,teamwork,collaboration,volleyball,nodejs,project,...,data,deep,php,databases,solution,dos,support,presentation,troubleshooting,detail
0,Medical Receptionist,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1,Clinical Research Associate,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Territory Sales Representative,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bus Driver,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Foreign Service Officer,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,Environmental Services Representative,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
218,Assistant General Manager,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
219,Operations Agent,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
220,Event Coordinator,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
#Now after creating dataframe 

#Now we have to create matrix

#converting dataframe into matrix
meta_data_matrix= np.array(meta_data)
meta_data_matrix

array([['Medical Receptionist', 0, 0, ..., 0, 0, 0],
       ['Clinical Research Associate', 1, 0, ..., 0, 0, 0],
       ['Territory Sales Representative', 0, 0, ..., 0, 0, 0],
       ...,
       ['Operations Agent', 0, 0, ..., 0, 0, 0],
       ['Event Coordinator', 1, 0, ..., 0, 0, 0],
       ['Mutual Fund Analyst', 1, 0, ..., 0, 0, 0]], dtype=object)

### Tanimoto/Jaccard Similarity
Calculating Union

In [44]:
def union(job1,job2):
    #Calculating Union
    result = list(set(job1) | set(job2))
    return result

#### Calculating Intersection

In [45]:
def intersection(job1,job2):
    #Calculating intersection
    result = list(set(job1) & set(job2))
    return result

#### Calculating similarity between jobs

In [46]:
def calculate_tanimoto_similarity():
    #Function to Calculate tanimoto similarity
    
    #Dictionary to store tanimoto similarity for each job with other in key:value format.
    dict_tanimoto_val={}
    for index, obj in enumerate(skills):
        # print(obj)
        inner_list=[]
        for next_index,next_obj in enumerate(skills):
            if(index==next_index):
                #because if job is same then 100% similarity
                inner_list.append(1.0)
                # continue
            
            else:
                # print(obj)
                union_result=union(obj, next_obj)
                # print("length of union result",len(union_result))
                intersection_result=intersection(obj, next_obj)
                # print("length of intersection result",len(intersection_result))

                inner_list.append(len(intersection_result)/len(union_result))

        dict_tanimoto_val[jobs[index]] = inner_list

     
    return dict_tanimoto_val
tanimoto_sim_dict=calculate_tanimoto_similarity()
print(tanimoto_sim_dict)

{'Medical Receptionist': [1.0, 0.0, 0.0, 0.0, 0.1935483870967742, 0.04, 0.1724137931034483, 0.17647058823529413, 0.13043478260869565, 0.0, 0.0, 0.0, 0.05555555555555555, 0.0, 0.05263157894736842, 0.09090909090909091, 0.0, 0.0, 0.0, 0.13636363636363635, 0.09523809523809523, 0.0, 0.047619047619047616, 0.038461538461538464, 0.14285714285714285, 0.0, 0.047619047619047616, 0.05555555555555555, 0.0, 0.0625, 0.0, 0.14285714285714285, 0.0, 0.6, 0.041666666666666664, 0.0, 0.0, 0.04, 0.0, 0.0, 0.0, 0.0, 0.0625, 0.0, 0.0, 0.06896551724137931, 0.1111111111111111, 0.1, 0.12, 0.05263157894736842, 0.09523809523809523, 0.21052631578947367, 0.1111111111111111, 0.0, 0.11538461538461539, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.09523809523809523, 0.11764705882352941, 0.0, 0.0, 0.0, 0.0, 0.0, 0.037037037037037035, 0.058823529411764705, 0.05263157894736842, 0.0, 0.0, 0.04, 0.2, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.11764705882352941, 0.0, 0.041666666666666664, 0.058823529411

### Recommending Top5 Jobs

#### Calculating similarity between skills query set and jobs skills set

In [47]:
def calulate_query_similarity(skills_query):

    tanimoto_dict={}

    for index,obj in enumerate(skills):
        union_result=union(skills_query, obj)
        # print("length of union result",len(union_result))
        intersection_result=intersection(skills_query, obj)
        # print("length of intersection result",len(intersection_result))

        tanimoto_dict[jobs[index]]=(len(intersection_result)/len(union_result))

    return tanimoto_dict

#### Finding similarity for query skills set and Top matched job

In [48]:
def recommend():
    #Function for Query preprocessing and calculation of jaccard coeff.

    skills_input=input("Please Enter Your Skills:  ")
    #enter queries with proper space in betwen

    #Deleting special charachter if any from input
    skills_preprocessed=delete_spec_chars(skills_input)

    #Tokenizing those skills
    tokens = word_tokenize(skills_preprocessed)

    #Converting all the skills in lower case, because while making dataframe also we converted skills in lower case
    tokens= [word.lower() for word in tokens]

    #removing stop words and words of length 1 also it may be possibility that user may enter some garbage value that's why
    skills_query=[word for word in tokens if word not in stop_words and len(word) > 1] #Removing stopwords 
    
    print(skills_query)

    #Calculating tanimoto similarity of skills with all the available job skills
    tanimoto_val=calulate_query_similarity(skills_query)

    # print(tanimoto_val)

    #Calculate highest matched documents 
    #for that we have to sort them in descending order
    sorted_d = dict( sorted(tanimoto_val.items(), key=operator.itemgetter(1),reverse=True))
    # top5=calculate_top(jaccard_val)
    # print(top5)
    # return top5

    return tanimoto_val,sorted_d

tanimoto_similarity_val,sorted_d=recommend()

print("before sorting")
print(tanimoto_similarity_val)

print("after sorting")
print(sorted_d)

['ai', 'collaboration']
before sorting
{'Medical Receptionist': 0.0, 'Clinical Research Associate': 0.0, 'Territory Sales Representative': 0.0, 'Bus Driver': 0.16666666666666666, 'Foreign Service Officer': 0.043478260869565216, 'Sales Analyst': 0.0, 'Strategy Consultant?': 0.05, 'Cost Engineer': 0.038461538461538464, 'Solutions Consultant - Electrical Engineering': 0.0, 'Power Regulator': 0.0, 'Conservation Technician': 0.0, 'Manager of Engineering': 0.0, 'Forecast Systems Manager': 0.0, 'Water Services Technician': 0.0, 'Senior Analyst Retail Analytics and Reporting': 0.0, 'Total Quality Management Director': 0.1, 'HVAC Mechanic': 0.0, 'Cognos Business Development Architect': 0.0, 'School Social Worker': 0.125, 'Treasurer': 0.0, 'Manufacturing Customer Service Representative': 0.0, 'Quality Assurance Manager': 0.0, 'Night Auditor': 0.125, 'Business Intelligence Developer': 0.07692307692307693, 'Solid Waste Field Technician': 0.05555555555555555, 'Healthcare Consultant': 0.125, 'Organi

In [49]:
def calculate_top(new_sorted_d):
    #Function to calculate top 5 documents according to the value of jaccard coeffecient.
    count=0
    top5=[]
    for item in new_sorted_d.keys():
        #Finding top5 documents

        if (count != 5):
            top5.append(item)
        else: 
            break
        count=count+1
    
    return top5

In [50]:
#Finding top matched job
top_match=list(sorted_d.keys())[0]
top_match

'Corporate Sales Manager'

In [51]:
#adding all the similarity values with top match with job in dictionary
new_dict={}
for index,obj in enumerate(tanimoto_sim_dict[top_match]):
    new_dict[jobs[index]]=obj
    
print(new_dict)

#now getting the value with highest match, i.e. we have to sort in descending order

new_sorted_d=dict( sorted(new_dict.items(), key=operator.itemgetter(1),reverse=True))

print("new sorted dict is:")
print("\n\n", new_sorted_d)
#as we can see values obtained are same as query so correct ans

#To retreive top 5 document:
top5=calculate_top(new_sorted_d)

print("top 5 recommended jobs are:")
print(top5)

{'Medical Receptionist': 0.0, 'Clinical Research Associate': 0.0, 'Territory Sales Representative': 0.0, 'Bus Driver': 0.16666666666666666, 'Foreign Service Officer': 0.043478260869565216, 'Sales Analyst': 0.0, 'Strategy Consultant?': 0.05, 'Cost Engineer': 0.038461538461538464, 'Solutions Consultant - Electrical Engineering': 0.0, 'Power Regulator': 0.0, 'Conservation Technician': 0.0, 'Manager of Engineering': 0.0, 'Forecast Systems Manager': 0.0, 'Water Services Technician': 0.0, 'Senior Analyst Retail Analytics and Reporting': 0.0, 'Total Quality Management Director': 0.1, 'HVAC Mechanic': 0.0, 'Cognos Business Development Architect': 0.0, 'School Social Worker': 0.125, 'Treasurer': 0.0, 'Manufacturing Customer Service Representative': 0.0, 'Quality Assurance Manager': 0.0, 'Night Auditor': 0.125, 'Business Intelligence Developer': 0.07692307692307693, 'Solid Waste Field Technician': 0.05555555555555555, 'Healthcare Consultant': 0.125, 'Organic Lab Worker': 0.125, 'Compliance Engin

In [52]:
#Top 5 Recommended jobs are

print(top5)

['Corporate Sales Manager', 'Threat Intelligence Analyst', 'Controller', 'Operations Clerk', 'Strength and Conditioning Coach']


### Euclidean Similarity

In [53]:
meta_data

,Jobs,ensemble,computing,electronics,wimax,teamwork,collaboration,volleyball,nodejs,project,...,data,deep,php,databases,solution,dos,support,presentation,troubleshooting,detail
0,Medical Receptionist,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1,Clinical Research Associate,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Territory Sales Representative,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bus Driver,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Foreign Service Officer,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,Environmental Services Representative,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
218,Assistant General Manager,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
219,Operations Agent,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
220,Event Coordinator,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
from sklearn.metrics.pairwise import euclidean_distances
     

eucdis = pd.DataFrame(euclidean_distances(meta_data.drop('Jobs', axis=1)),index=meta_data['Jobs'], columns = meta_data['Jobs'])
eucdis

Jobs,Medical Receptionist,Clinical Research Associate,Territory Sales Representative,Bus Driver,Foreign Service Officer,Sales Analyst,Strategy Consultant?,Cost Engineer,Solutions Consultant - Electrical Engineering,Power Regulator,...,Sales and Marketing Coordinator,Environmental Project Analyst,International Travel Consultant,Airline Pilot,Senior Competitive Intelligence Manager,Environmental Services Representative,Assistant General Manager,Operations Agent,Event Coordinator,Mutual Fund Analyst
Jobs,,,,,,,,,,,,,,,,,,,,,
Medical Receptionist,0.000000,4.242641,4.123106,4.472136,5.000000,4.898979,4.898979,5.291503,4.472136,4.242641,...,4.123106,4.358899,4.242641,4.123106,4.000000,4.690416,4.795832,4.123106,4.358899,4.358899
Clinical Research Associate,4.242641,0.000000,1.732051,2.449490,4.582576,3.464102,4.472136,5.291503,3.741657,2.000000,...,1.732051,3.000000,2.449490,1.732051,1.414214,3.162278,3.605551,1.732051,2.236068,1.732051
Territory Sales Representative,4.123106,1.732051,0.000000,2.236068,4.690416,3.000000,4.123106,5.000000,3.605551,1.732051,...,1.414214,2.828427,2.236068,1.414214,1.732051,2.645751,3.162278,1.414214,2.449490,2.000000
Bus Driver,4.472136,2.449490,2.236068,0.000000,4.795832,3.162278,4.242641,5.099020,3.741657,1.414214,...,2.236068,3.316625,2.449490,2.236068,2.449490,3.464102,3.605551,2.236068,2.645751,2.236068
Foreign Service Officer,5.000000,4.582576,4.690416,4.795832,0.000000,5.000000,5.196152,5.744563,5.000000,4.582576,...,4.472136,4.690416,4.582576,4.690416,4.795832,4.795832,5.291503,4.690416,4.898979,4.898979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Environmental Services Representative,4.690416,3.162278,2.645751,3.464102,4.795832,3.741657,4.690416,5.099020,4.242641,3.162278,...,3.000000,3.605551,3.162278,3.000000,2.828427,0.000000,3.605551,3.000000,3.605551,3.316625
Assistant General Manager,4.795832,3.605551,3.162278,3.605551,5.291503,3.605551,5.000000,5.196152,4.582576,3.316625,...,3.464102,4.000000,3.872983,3.464102,3.316625,3.605551,0.000000,3.464102,4.000000,3.464102
Operations Agent,4.123106,1.732051,1.414214,2.236068,4.690416,3.316625,4.358899,5.196152,3.605551,1.732051,...,1.414214,2.449490,2.236068,1.414214,1.732051,3.000000,3.464102,0.000000,2.000000,1.414214


In [55]:
euc_job_dict = eucdis.to_dict()
euc_job_dict

{'Medical Receptionist': {'Medical Receptionist': 0.0,
  'Clinical Research Associate': 4.242640687119285,
  'Territory Sales Representative': 4.123105625617661,
  'Bus Driver': 4.47213595499958,
  'Foreign Service Officer': 5.0,
  'Sales Analyst': 4.898979485566356,
  'Strategy Consultant?': 4.898979485566356,
  'Cost Engineer': 5.291502622129181,
  'Solutions Consultant - Electrical Engineering': 4.47213595499958,
  'Power Regulator': 4.242640687119285,
  'Conservation Technician': 4.0,
  'Manager of Engineering': 4.242640687119285,
  'Forecast Systems Manager': 4.123105625617661,
  'Water Services Technician': 4.123105625617661,
  'Senior Analyst Retail Analytics and Reporting': 4.242640687119285,
  'Total Quality Management Director': 4.47213595499958,
  'HVAC Mechanic': 4.123105625617661,
  'Cognos Business Development Architect': 4.123105625617661,
  'School Social Worker': 4.69041575982343,
  'Treasurer': 4.358898943540674,
  'Manufacturing Customer Service Representative': 4.35

In [56]:
import math
import re
from collections import Counter

In [57]:
WORD = re.compile(r"\w+")

In [58]:
def get_euclidean(vec1, vec2):
    
    union = list(set(set(vec1.keys()) | set(vec2.keys())))
    #print(union)
    euc1 = []
    euc2 = []
    for u in union:
      if u in vec1.keys():
        euc1.append(1)
      else:
        euc1.append(0) 

    for u in union:
      if u in vec2.keys():
        euc2.append(1)
      else:
        euc2.append(0) 
   # euc_final=sum(list(abs(np.array(euc1) - np.array(euc2)))) 
    summ = 0
    for x in euc1:
      for y in euc2:
        summ += math.pow((x-y),2)
    res = math.sqrt(summ)   
   # euc_final=math.sqrt(sum(list(math.pow((np.array(euc1) - np.array(euc2)),2))))
    return res       
   

In [59]:
def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


### Example how euclidean based Similarity works

In [60]:
text = "I am Jhon Kumar"
vector = text_to_vector(text)
vector.values()
vector.keys()

dict_keys(['I', 'am', 'Jhon', 'Kumar'])

In [61]:
text2 = "I am Richa"
vector2 = text_to_vector(text2)
#vector2.keys()
get_euclidean(vector, vector2)

3.3166247903554

In [62]:
def euclidean_recommend():
    #Function for Query preprocessing and calculation of jaccard coeff.

    skills_input=input("Please Enter Your Skills:  ")
    #enter queries with proper space in betwen

    #Deleting special charachter if any from input
    skills_preprocessed=delete_spec_chars(skills_input)

    #Tokenizing those skills
    tokens = word_tokenize(skills_preprocessed)

    #Converting all the skills in lower case, because while making dataframe also we converted skills in lower case
    tokens= [word.lower() for word in tokens]

    #removing stop words and words of length 1 also it may be possibility that user may enter some garbage value that's why
    skills_query=[word for word in tokens if word not in stop_words and len(word) > 1] #Removing stopwords 
    query_txt = delete_spec_chars(str(skills_query))
    #print(skills_query)
    #print(query_txt)

    qvector = text_to_vector(query_txt)

    euclidean_dict = {}

    for ind,ski in enumerate(new_skills):
      ski_txt = delete_spec_chars(str(ski))
      ski_vector = text_to_vector(ski_txt)
      euclidean = get_euclidean(qvector, ski_vector)
      euclidean_dict[jobs[ind]]=euclidean
   
    sorted_euclidean = dict( sorted(euclidean_dict.items(), key=operator.itemgetter(1),reverse=False))
    return euclidean_dict, sorted_euclidean

euc_dict, sorted_dict = euclidean_recommend()
#euclidean query similarity
print(euc_dict)
#sorted euclidean query similarity
print(sorted_dict)

{'Medical Receptionist': 15.132745950421556, 'Clinical Research Associate': 2.8284271247461903, 'Territory Sales Representative': 2.0, 'Bus Driver': 3.872983346207417, 'Foreign Service Officer': 21.540659228538015, 'Sales Analyst': 10.583005244258363, 'Strategy Consultant?': 18.547236990991408, 'Cost Engineer': 25.079872407968907, 'Solutions Consultant - Electrical Engineering': 11.180339887498949, 'Power Regulator': 2.8284271247461903, 'Conservation Technician': 2.23606797749979, 'Manager of Engineering': 3.605551275463989, 'Forecast Systems Manager': 3.7416573867739413, 'Water Services Technician': 2.8284271247461903, 'Senior Analyst Retail Analytics and Reporting': 4.69041575982343, 'Total Quality Management Director': 9.219544457292887, 'HVAC Mechanic': 2.8284271247461903, 'Cognos Business Development Architect': 2.8284271247461903, 'School Social Worker': 7.280109889280518, 'Treasurer': 10.198039027185569, 'Manufacturing Customer Service Representative': 8.246211251235321, 'Qualit

In [63]:
euc_top_match=list(sorted_dict.keys())[0]
euc_top_match

'Dermatology Nurse'

In [64]:
new_euc_dict = euc_job_dict[top_match]
print(new_euc_dict)

{'Medical Receptionist': 4.123105625617661, 'Clinical Research Associate': 2.23606797749979, 'Territory Sales Representative': 2.0, 'Bus Driver': 2.23606797749979, 'Foreign Service Officer': 4.69041575982343, 'Sales Analyst': 3.605551275463989, 'Strategy Consultant?': 4.358898943540674, 'Cost Engineer': 5.0, 'Solutions Consultant - Electrical Engineering': 3.605551275463989, 'Power Regulator': 2.23606797749979, 'Conservation Technician': 1.7320508075688772, 'Manager of Engineering': 2.23606797749979, 'Forecast Systems Manager': 2.449489742783178, 'Water Services Technician': 2.0, 'Senior Analyst Retail Analytics and Reporting': 2.6457513110645907, 'Total Quality Management Director': 3.0, 'HVAC Mechanic': 2.0, 'Cognos Business Development Architect': 2.0, 'School Social Worker': 2.6457513110645907, 'Treasurer': 3.4641016151377544, 'Manufacturing Customer Service Representative': 3.1622776601683795, 'Quality Assurance Manager': 2.6457513110645907, 'Night Auditor': 2.6457513110645907, 'B

In [65]:
new_sorted_euc_d=dict( sorted(new_euc_dict.items(), key=operator.itemgetter(1),reverse=False))
print(new_sorted_euc_d)

{'Corporate Sales Manager': 0.0, 'Controller': 1.0, 'Operations Clerk': 1.0, 'Threat Intelligence Analyst': 1.0, 'Strength and Conditioning Coach': 1.0, 'Computer Science Teacher': 1.0, 'Patio Room Installer': 1.0, 'Process Control Engineer': 1.0, 'Chief of Career Pathways and Integrated Learning': 1.4142135623730951, 'Assistant Coach': 1.4142135623730951, 'Human Resources Assistant': 1.4142135623730951, 'Retail Sales Associate': 1.4142135623730951, 'Dermatology Nurse': 1.4142135623730951, 'Vice President of Engineering': 1.4142135623730951, 'Mining Safety Engineer': 1.4142135623730951, 'Field Technician': 1.4142135623730951, 'Laboratory Assistant': 1.4142135623730951, 'Vice Principal': 1.4142135623730951, 'Conservation Technician': 1.7320508075688772, 'Roadie': 1.7320508075688772, 'Travel Consultant': 1.7320508075688772, 'Healthcare Business Intelligence Manager': 1.7320508075688772, 'Quality Assurance Technologist': 1.7320508075688772, 'Human Resources Director': 1.7320508075688772, 

In [66]:
calculate_top(new_sorted_euc_d)

['Corporate Sales Manager',
 'Controller',
 'Operations Clerk',
 'Threat Intelligence Analyst',
 'Strength and Conditioning Coach']

### Performance Analysis of Euclidean Distance

In [67]:
test_data = pd.read_csv(r'C:\Users\Prajwal\Desktop\Vyzen Assessment\Job Recommendation\datasets\test.csv')
test_data.head()

,user_id,job_title,skills
0,103,RF Engineer,"{'skills': {'training': 2, 'implementation': 1..."
1,48,Assistant Technician,"{'skills': {'consulting': 1, 'communication': ..."
2,6,Strategy Consultant?,"{'skills': {'mobile': 1, 'communication': 2, '..."
3,72,Technical Writer,"{'skills': {'documentation': 1, 'dom': 1}}"
4,153,Front Desk Clerk,"{'skills': {'communication': 2, 'collaboration..."


In [68]:
test_jobs = test_data['job_title'].unique()
# print(test_jobs)
print(len(test_jobs))

93


In [69]:
#creating list of dictionary to store the job and it's corresponding skills
test_skills = []
for j in test_jobs:
    #traversing through job and comparing it with job title in dataframe if both are same then adding their skills.

    #creating inner list to store the skills of one job
    d=[]
    for i, row in test_data.iterrows():
        if(row['job_title']==j):
            d.append(row['skills'])
    print(d)

  
    #removing all the special characters from that
    doc = delete_spec_chars(str(d))

    #removing digits from that
    doc = re.sub(r'\d+','',doc)

    #removing skills word from that
    doc=re.sub(r'skills','',doc)

    #Applying word tokenization to extract tokens
    tokens = word_tokenize(doc) 
    print("document is: ")
    print(doc)
    print(" tokens are: ")

    tokens= [word.lower() for word in tokens]
    print(tokens)
    
    #To remove duplicate skills 
    tokens=list(set(tokens))

    #Adding that in skills dicitonary for one job
    test_skills.append(tokens)

#   skillss.append(newdf.loc[newdf['job_title'] == j]['skills'])
  
print(test_skills)

["{'skills': {'training': 2, 'implementation': 1, 'communication': 1, 'reporting': 2, 'project manager': 1, 'analytics': 1, 'technical': 3}}"]
document is: 
 training  implementation  communication  reporting  project manager  analytics  technical 
 tokens are: 
['training', 'implementation', 'communication', 'reporting', 'project', 'manager', 'analytics', 'technical']
["{'skills': {'consulting': 1, 'communication': 1, 'programming': 1, 'analytics': 1, 'analytical': 1, 'mathematics': 1, 'data analytics': 1}}"]
document is: 
 consulting  communication  programming  analytics  analytical  mathematics  data analytics 
 tokens are: 
['consulting', 'communication', 'programming', 'analytics', 'analytical', 'mathematics', 'data', 'analytics']
["{'skills': {'mobile': 1, 'communication': 2, 'collaboration': 2, 'training': 2, 'e-commerce': 1}}", "{'skills': {'training': 1, 'data collection': 1, 'processing': 3, 'implementation': 1, 'communication': 1, 'data mining': 2, 'innovation': 1, 'machine

In [70]:
print(test_skills[0])
print(len(test_skills))

['training', 'implementation', 'analytics', 'technical', 'reporting', 'manager', 'project', 'communication']
93


In [71]:
test_skills_strings = []
for s in range(len(test_skills)):
  qtext = delete_spec_chars(str(test_skills[s]))
  test_skills_strings.append(qtext)
test_skills_strings

['training implementation analytics technical reporting manager project communication',
 'data programming analytics consulting mathematics analytical communication',
 'implementation mobile mathematics technical collaboration engineering communication training mining machine design processing go ecommerce learning data collection deep innovation',
 'documentation dom',
 'communication collaboration ensemble',
 'internet analytics collaboration ecommerce communication',
 'mobile design coaching hardware reporting',
 'training implementation programming mentoring presentation innovation',
 'leadership training implementation internet technical coaching go ecommerce project management',
 'training collaboration ai tableau communication',
 'ensemble implementation design processing programming technical ecommerce reporting',
 'internet',
 'ensemble design modeling programming collaboration ai engineering matlab analytical communication',
 'internet communication',
 'ecommerce',
 'communic

In [72]:
def euclidean_recommend1(skills_input):
    #Function for Query preprocessing and calculation of jaccard coeff.

    #skills_input=input("Please Enter Your Skills:  ")
    #enter queries with proper space in betwen

    #Deleting special charachter if any from input
    skills_preprocessed=delete_spec_chars(skills_input)

    #Tokenizing those skills
    tokens = word_tokenize(skills_preprocessed)

    #Converting all the skills in lower case, because while making dataframe also we converted skills in lower case
    tokens= [word.lower() for word in tokens]

    #removing stop words and words of length 1 also it may be possibility that user may enter some garbage value that's why
    skills_query=[word for word in tokens if word not in stop_words and len(word) > 1] #Removing stopwords 
    query_txt = delete_spec_chars(str(skills_query))
    #print(skills_query)
    #print(query_txt)

    qvector = text_to_vector(query_txt)

    euclidean_dict = {}

    for ind,ski in enumerate(new_skills):
      ski_txt = delete_spec_chars(str(ski))
      ski_vector = text_to_vector(ski_txt)
      euclidean = get_euclidean(qvector, ski_vector)
      euclidean_dict[jobs[ind]]=euclidean

    sorted_euclidean = dict( sorted(euclidean_dict.items(), key=operator.itemgetter(1),reverse=False))
    return euclidean_dict, sorted_euclidean

In [73]:
def check_euclidean_performance(test_skills_strings):
  top_jobs = []
  for st in test_skills_strings:
    #print(st)
    euc_dict, sorted_dict = euclidean_recommend1(st)
    euc_top_match=list(sorted_dict.keys())[0]    
    #print(euc_top_match)
    new_euc_dict = euc_job_dict[euc_top_match]
    abc=dict( sorted( new_euc_dict.items(), key=operator.itemgetter(1),reverse=False))
    #print(abc)
    result = calculate_top(abc)
    top_jobs.append(result)
    #print()
    #print()

  return top_jobs


In [74]:
output_test1 = check_euclidean_performance(test_skills_strings)
output_test1

[['RF Engineer',
  'Material Handler',
  'Cruise Ship',
  'Pipeline Engineer',
  'Operations Agent'],
 ['Assistant Technician',
  'Quality Assurance Technologist',
  'Oceanographer',
  'Special Education Teacher',
  'Instrumentation Engineer'],
 ['Strategy Consultant?',
  'Golf Pro',
  'Countertop Fabricator and Installer',
  'Commercial Sales Representative',
  'Power Regulator'],
 ['Maintenance Engineer',
  'Technical Writer',
  'Assistant Coach',
  'School Nurse',
  'Conservation Technician'],
 ['Marketing Consultant',
  'Front Desk Clerk',
  'Cook',
  'Retail Sales Associate',
  'Drilling Engineer'],
 ['Assistant Customer Care Center Manager',
  'Associate Athletic Director',
  'Territory Sales Representative',
  'Sales Coordinator',
  'Retail Sales Associate'],
 ['Quality Assurance Manager',
  'Senior Manager of Industry Analytics',
  'Reservoir Engineer',
  'Senior Manager Product Intelligence and Cost Analytics',
  'Laboratory Sales Consultant'],
 ['Ground Support Equipment Mech

In [75]:

predval = []
for ii,output in enumerate(output_test1):
  #test_data
  user_job = test_jobs[ii]
  if (user_job in output):
    predval.append(1)
  else:
    predval.append(0)
print(len(predval))
print(predval)

93
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [76]:
accuracy = sum(predval)/len(predval)

In [77]:
print(accuracy)

0.8387096774193549


In [78]:
error_rate=predval.count(0)/len(predval)
error_rate

0.16129032258064516

#### Using Accurate Job recommendation list

In [79]:
def check_euclidean_performance_accurate(test_skills_strings):
  top_jobs_accurate = []
  for st in test_skills_strings:
    #print(st)
    euc_dict, sorted_dict = euclidean_recommend1(st)
    euc_top_match=list(sorted_dict.keys())[0]    
    value = sorted_dict[euc_top_match]
    accvals = []
    for xx,yy in sorted_dict.items():
      if xx == euc_top_match:
        accvals.append(xx)
      else:
        if yy == value:
          accvals.append(xx)
        else:
          break
    # if len(accvals) >= 5:
    top_jobs_accurate.append(accvals)
    # else:
    #   new_man_dict = man_job_dict[man_top_match]
    #   abc=dict( sorted( new_man_dict.items(), key=operator.itemgetter(1),reverse=False))
    #   result = calculate_top(abc)
    #   top_jobs_accurate.append(result)
    

  return top_jobs_accurate

In [80]:
output_test2 = check_euclidean_performance_accurate(test_skills_strings)
output_test2

[['RF Engineer'],
 ['Assistant Technician'],
 ['Strategy Consultant?'],
 ['Maintenance Engineer', 'Technical Writer'],
 ['Marketing Consultant', 'Front Desk Clerk', 'Cook'],
 ['Assistant Customer Care Center Manager', 'Associate Athletic Director'],
 ['Quality Assurance Manager',
  'Senior Manager of Industry Analytics',
  'Reservoir Engineer',
  'Senior Manager Product Intelligence and Cost Analytics',
  'Laboratory Sales Consultant'],
 ['Ground Support Equipment Mechanic'],
 ['Treasurer'],
 ['International Travel Consultant'],
 ['Market Access Associate'],
 ['Travel Consultant',
  'Human Resources Director',
  'Travel Secretary',
  'Firmware Engineer',
  'Front End Load Driver',
  'Assistant Principal',
  'Integration/Business Intelligence Technical Lead',
  'Manager Special Events',
  'Customer Service Agent'],
 ['Software Developer'],
 ['Sales Coordinator',
  'Athletic Scout',
  'Senior Mechanical Engineer',
  'Commercial Print Management Consultant',
  'Business Objects Architect'

In [81]:
predval2 = []
for iii,outputt in enumerate(output_test2):
  #test_data
  user_job = test_jobs[iii]
  if (user_job in outputt):
    predval2.append(1)
  else:
    predval2.append(0)
print(len(predval2))
print(predval2)

93
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [82]:
accuracy2 = sum(predval2)/len(predval2)
print(accuracy2)
error_rate2=predval2.count(0)/len(predval2)
print(error_rate2)

0.8494623655913979
0.15053763440860216


### Enhanced Recommendations with User Feedback
Enhanced recommendations with user feedback aim to refine job recommendations by incorporating user preferences, making them more personalized and relevant.

In [84]:
feedback_data = []

def get_user_feedback():
    feedback = input("Please provide feedback (job_title:rating), or type 'stop' to end: ")
    if feedback.lower() == 'stop':
        return False
    feedback_items = feedback.split(',')
    for item in feedback_items:
        job_title, rating = item.split(':')
        feedback_data.append((job_title.strip(), float(rating.strip())))
    return True

def refine_recommendations():
    global skills
    while True:
        euc_dict, sorted_dict = euclidean_recommend()
        print("Euclidean query similarity:", euc_dict)
        print("Sorted Euclidean query similarity:", sorted_dict)

        if not get_user_feedback():
            print("Feedback session ended.")
            break
        
        for job_title, rating in feedback_data:
            if rating > 3:  # Assuming a rating scale from 1 to 5
                matched_jobs = df_combined2[df_combined2['job_title'] == job_title]
                if not matched_jobs.empty:
                    job_skills = matched_jobs['skills'].values[0]
                    job_skills = eval(job_skills)
                    skills.append(job_skills)
                else:
                    print(f"No matching job found for title: {job_title}")
        
        feedback_data.clear()  # Clear feedback data after processing
        print("Updated recommendations based on feedback:")

# Start recommendation refinement loop
refine_recommendations()

Euclidean query similarity: {'Medical Receptionist': 15.033296378372908, 'Clinical Research Associate': 3.1622776601683795, 'Territory Sales Representative': 2.23606797749979, 'Bus Driver': 4.47213595499958, 'Foreign Service Officer': 22.02271554554524, 'Sales Analyst': 11.045361017187261, 'Strategy Consultant?': 19.026297590440446, 'Cost Engineer': 25.019992006393608, 'Solutions Consultant - Electrical Engineering': 11.045361017187261, 'Power Regulator': 3.1622776601683795, 'Conservation Technician': 1.4142135623730951, 'Manager of Engineering': 3.1622776601683795, 'Forecast Systems Manager': 4.123105625617661, 'Water Services Technician': 2.23606797749979, 'Senior Analyst Retail Analytics and Reporting': 5.0990195135927845, 'Total Quality Management Director': 9.055385138137417, 'HVAC Mechanic': 2.23606797749979, 'Cognos Business Development Architect': 2.23606797749979, 'School Social Worker': 7.0710678118654755, 'Treasurer': 10.04987562112089, 'Manufacturing Customer Service Repres

In [89]:
def refine_recommendations():
    global skills
    user_feedback = {}

    while True:
        euc_dict, sorted_dict = recommend()
        print("Euclidean query similarity:", euc_dict)
        print("Sorted Euclidean query similarity:", sorted_dict)

        feedback = input("Please provide feedback in the format 'job_title:relevant/irrelevant', or type 'stop' to end: ")
        if feedback.lower() == 'stop':
            print("Feedback session ended.")
            break
        
        feedback_items = feedback.split(',')
        for item in feedback_items:
            job_title, relevance = item.split(':')
            user_feedback[job_title.strip()] = relevance.strip().lower()

        relevant_jobs = [job for job, relevance in user_feedback.items() if relevance == 'relevant']
        irrelevant_jobs = [job for job in sorted_dict.keys() if job not in relevant_jobs]

        if not relevant_jobs:
            print("No relevant jobs selected. Continuing with current recommendations.")
            continue

        # Update the skills list based on feedback
        for job_title in relevant_jobs:
            matched_jobs = meta_data[meta_data['job_title'] == job_title]
            if not matched_jobs.empty:
                job_skills = matched_jobs['skills'].values[0]
                job_skills = eval(job_skills)
                if job_skills not in skills:
                    skills.append(job_skills)
            else:
                print(f"No matching job found for title: {job_title}")

        print("Refining recommendations based on feedback.")

# Start recommendation refinement loop
refine_recommendations()

['analytics']
Euclidean query similarity: {'Medical Receptionist': 0.0, 'Clinical Research Associate': 0.0, 'Territory Sales Representative': 0.0, 'Bus Driver': 0.0, 'Foreign Service Officer': 0.0, 'Sales Analyst': 0.0, 'Strategy Consultant?': 0.0, 'Cost Engineer': 0.0, 'Solutions Consultant - Electrical Engineering': 0.0, 'Power Regulator': 0.0, 'Conservation Technician': 0.0, 'Manager of Engineering': 0.0, 'Forecast Systems Manager': 0.0, 'Water Services Technician': 0.0, 'Senior Analyst Retail Analytics and Reporting': 0.0, 'Total Quality Management Director': 0.0, 'HVAC Mechanic': 0.0, 'Cognos Business Development Architect': 0.0, 'School Social Worker': 0.0, 'Treasurer': 0.0, 'Manufacturing Customer Service Representative': 0.0, 'Quality Assurance Manager': 0.0, 'Night Auditor': 0.0, 'Business Intelligence Developer': 0.0, 'Solid Waste Field Technician': 0.058823529411764705, 'Healthcare Consultant': 0.0, 'Organic Lab Worker': 0.0, 'Compliance Engineer': 0.0, 'Chief of Career Path